In [1]:
import sys
import os
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
%run dataset.ipynb 
%run neuralnet.ipynb
%run training.ipynb

Using TensorFlow backend.


In [27]:
%run getgraph.ipynb
# Test section
graph_path = './graph.pb'
# inspectGraph(graph_path)
import tensorflow as tfq
g = tf.GraphDef()
g.ParseFromString(open(graph_path, "rb").read())

# print([n for n in g.node if n.name.find("input") != -1])
print([n for n in g.node if n.name.find("output") != -1])

[]


In [3]:
# Configuration parameters
parser = argparse.ArgumentParser(usage='A training program for classifying the EMNIST dataset')
parser.add_argument('-f', '--file', type=str, help='Path .mat file data', required=True)
parser.add_argument('--width', type=int, default=28, help='Width of the images')
parser.add_argument('--height', type=int, default=28, help='Height of the images')
parser.add_argument('--max', type=int, default=None, help='Max amount of data to use')
parser.add_argument('--epochs', type=int, default=5, help='Number of epochs to train on')
parser.add_argument('--verbose', action='store_true', default=False, help='Enables verbose printing')

#     args = parser.parse_args()

data_file = 'emnist-byclass.mat'
data_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/data/' + data_file
#     args = parser.parse_args(['-f', data_path, '--max', '500'])
args = parser.parse_args(['-f', data_path])

bin_dir = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin'
if not os.path.exists(bin_dir):
    os.makedirs(bin_dir)

In [4]:
# Construct training data
training_data = load_data(args.file, width=args.width, height=args.height, max_data=args.max, verbose=args.verbose)

In [5]:
# Construct model
model = build_net(training_data, width=args.width, height=args.height, verbose=args.verbose)

In [6]:
# Train
train(model, training_data, epochs=1)

Train on 697932 samples, validate on 116323 samples
Epoch 1/1
697932/697932 [==============================] - 2201s - loss: 0.6171 - acc: 0.8032 - val_loss: 0.4130 - val_acc: 0.8514
Test loss: 0.413009618327
Test accuracy: 0.8514481229


In [25]:
version = 0

model_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin/v' + str(version) + '/model.h5'

(model, output_graph_def) = extractgraph(model_path)


In [17]:
(x_train, y_train), (x_test, y_test), mapping, nb_classes = training_data
    
y_test = np_utils.to_categorical(y_test, nb_classes)

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.356566239601
Test accuracy: 0.869819382238


In [41]:
# Generating labels file
labels = ''
for i in range(68):
    labels += ((chr(i+55) if i > 9 else str(i)) + ('\n' if i < 67 else '')) if i < 36 or i > 41 else ''
    
# print(labels)
with open("labels.txt", "w") as label_file:
    label_file.write(labels)
